## 활성화 함수

### 시그모이드(Sigmoid) 함수

- 시그모이드 함수는 선형 함수의 결과를 0 ~ 1 사이에서 비선형 형태로 변형한다.

- 주로 로지스틱 회귀와 같은 분류 문제를 확률적으로 표현하는데 사용한다.

- 과거에는 인기가 많았으나, 딥러닝 모델의 깊이가 깊어지면 기울기가 사라지는 <font color="orange">'기울기 소멸 문제(vanishing gradient problem)'</font>가 발생하여 딥러닝 모델에서는 잘 사용하지 않는다.

- 시그모이드는 다음 수식을 사용한다.

$$ f(x) = \frac{1}{1 + e^{-x}} $$

#### <font color="orange">기울기 소멸 문제</font>

![Image](https://github.com/user-attachments/assets/77fd637c-3cf1-4040-93c2-440e8161bc58)

![Image](https://github.com/user-attachments/assets/13685579-7d09-4f76-ab76-25227cb26b3d)

- 시그모이드 함수 양 끝부분에서 기울기를 구하게 되면 0 혹은 0에 가까운 아주 작은 값이 나온다.

- Back Propagation 과정에서 시그모이드 함수에서의 기울기를 곱하게 된다.

![Image](https://github.com/user-attachments/assets/da588242-67b0-4c5d-90b3-c853aea788d4)

- 딥러닝 모델의 깊이가 깊어지게 되면 0에 가까운 기울기가 계속해서 곱해지게 되어 모델의 앞 부분에서는 거의 기울기를 전파 받을 수 없게 된다.

### 렐루(ReLU) 함수

- 최근 활발히 사용되는 렐루 함수는 입력(x)이 음수일 때는 0을 출력하고, 양수일 때는 x를 출력한다.

- 경사 하강법(gradient descent)에 영향을 주지 않아 학습 속도가 빠르고, 기울기 소멸 문제가 발생하지 않는 장점이 있다.

- 렐루 함수는 일반적으로 은닉층에서 사용되며, 하이퍼볼릭 탄젠트 함수 대비 학습 속도가 6배 빠르다.

- 문제는 음수 값을 입력받으면 항상 0을 출력하기 때문에 학습 능력이 감소한다.

![Image](https://github.com/user-attachments/assets/47fa592d-d393-4496-a718-44b181eac4fd)

### 소프트맥스(Softmax) 함수

- 소프트맥스 함수는 입력 값을 0~1 사이에 출력되도록 정규화하여 출력 값들의 총합이 항상 1이 되도록 한다.

- 소프트맥스 함수는 보통 딥러닝에서 출력 노드의 활성화 함수로 많이 사용한다.

- 수식으로 표현하면 다음과 같다.
$$
y_k = \frac{exp(a_k)}{\sum_{i=1}^n exp(a_i)}
$$

- n은 출력층의 뉴런 개수, $y_k$는 그 중 k번째 출력을 의미한다.

- 즉, 이 수식처럼 소프트맥스 함수의 분자는 입력 신호 $a_k$의 지수 함수, 분모는 모든 입력 신호의 지수 함수 합으로 구성된다.

### MNIST softmax

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# MNIST data image of shape 28 * 28 = 784
linear = torch.nn.Linear(784, 10, bias=True).to(device)

In [7]:
# initialization
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-1.2901,  0.3930, -0.1891,  ..., -0.1626, -0.3275,  0.5608],
        [-1.6757, -0.2454,  0.9221,  ...,  0.9559,  0.6160, -0.4226],
        [ 0.2444,  1.4810, -2.0906,  ...,  0.1206,  1.0044, -0.3737],
        ...,
        [ 0.6234,  1.8019, -2.7563,  ..., -0.5889, -0.5576,  0.7360],
        [-0.2871, -1.3313, -2.2248,  ...,  0.0309,  0.9180, -0.1482],
        [ 0.7678,  0.6624, -0.5362,  ...,  0.2338,  0.3688, -0.7182]],
       device='cuda:0', requires_grad=True)

In [8]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

In [9]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost.item()
    
    avg_cost /= total_batch

    print(f'Epoch: {epoch + 1:04d}, cost = {avg_cost:.9f}')
    
print('Learning finished')

Epoch: 0001, cost = 5.645694694
Epoch: 0002, cost = 1.696827326
Epoch: 0003, cost = 1.121626605
Epoch: 0004, cost = 0.882779920
Epoch: 0005, cost = 0.748860329
Epoch: 0006, cost = 0.661943850
Epoch: 0007, cost = 0.599809842
Epoch: 0008, cost = 0.553705251
Epoch: 0009, cost = 0.517529079
Epoch: 0010, cost = 0.488772514
Epoch: 0011, cost = 0.465197288
Epoch: 0012, cost = 0.445370481
Epoch: 0013, cost = 0.428630506
Epoch: 0014, cost = 0.413612143
Epoch: 0015, cost = 0.401215988
Learning finished


In [22]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    
    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8894999623298645
Label:  8
Prediction:  8


### MNIST ReLU

In [11]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [13]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [14]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [15]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [16]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [17]:
# Initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [18]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [19]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost.item()
    
    avg_cost /= total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 128.860448430
Epoch: 0002 cost = 35.916095277
Epoch: 0003 cost = 23.048260633
Epoch: 0004 cost = 16.049220863
Epoch: 0005 cost = 11.694718062
Epoch: 0006 cost = 8.532821981
Epoch: 0007 cost = 6.454253663
Epoch: 0008 cost = 4.768246512
Epoch: 0009 cost = 3.631305532
Epoch: 0010 cost = 2.853135378
Epoch: 0011 cost = 2.093369654
Epoch: 0012 cost = 1.688200809
Epoch: 0013 cost = 1.290744972
Epoch: 0014 cost = 1.022304224
Epoch: 0015 cost = 0.780164239
Learning finished


In [21]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9460999965667725
Label:  9
Prediction:  9
